# **Estequiometría**

Resolvedor para combustión completa, obtenido de [link text](https://chemicals.readthedocs.io/chemicals.combustion.html#heat-of-combustion-and-stiochiometry).

Versión: 0.1.
Fecha: 2022-06-25.
Realizó: PMB.

Concepto:

Si bien me gustaría que pudiera computar el argón, vale la pena tener lista la herramienta, aunque sea simple, para cálculos sencillos. También puedo tener las dos herramientas en paralelo.

Mejoras para realizar:

1) Reescribir para que pueda tomar al Argón en la cuenta. Sea modificando este, haciendo un parche, o reescribiendo la estequiometría desde cero.

1bis) Hacer un conversor de fracciones molares a másicas, y de másicas a molares. Para esto debemos cambiar la forma en la que ingresamos los datos, y además debemos precalcular la masa molar, tanto del aire como del gas natural.

2) Reescribir para tomar datos de un Excel. Composiciones molares y caudales molares. Pensar bien las formas en que le cargo los datos.

3) Reescribir para volcar datos en un Excel. Composiciones molares y caudales molares. Ver cómo genero distintas salidas.

4) Incorporar los siguientes componentes:

- Argón
- H2 (habilitar para combustión incompleta, como parte del combustible ya figura.)
- CO (habilitar para combustión incompleta, como parte del combustible ya figura.)
- NOx (habilitar para combustión incompleta)

5) Hacer una estequiometría para combustibles líquidos y sólidos, en donde la forma de expresar la composición es distinta.

---

# *Importación de librerías*


En primer lugar, con !pip instalamos los paquetes si Colab no los toma. Luego, debemos importar la librería chemicals, y lo hacemos con un alias. La librería cirpy la importamos sin alias.

In [2]:
import chemicals as ch
import cirpy
import numpy as np

Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.


# *Consulta de CAS*

Con esto podemos hacer consultas sobre el CAS. Sin embargo, da varios valores, algunos 'deprecados' (obsoletos).

In [3]:
cirpy.resolve('H2O', 'cas')

['14314-42-2', '558440-22-5', '13670-17-2', '7732-18-5', '17778-80-2']

# *Ingreso de datos*

Ahora, debemos listar todos los elementos que participan de la reacción. Sea que estén en el combustible (fuel) o en el aire.

De cada uno de ellos debemos dar el [identificador CAS](https://commonchemistry.cas.org/) y la cantidad de átomos de la molécula.

Otros links útiles son:

1.   [Springer](https://materials.springer.com/substanceprofile/docs/smsid_flvtyceefwhjkvfw)
2.   [PubChem](https://pubchem.ncbi.nlm.nih.gov/compound/Oxygen#section=CAS)
3.   [rsc](https://www.rsc.org/periodic-table/element/8/oxygen)
4.   [echa.europa](https://echa.europa.eu/substance-information/-/substanceinfo/100.029.051)


Aquí ingresamos las fracciones molares del aire y del combustible. Aunque ciertos elementos no estén presentes en el aire, debemos incluirlos en la composición molar, porque así lo pide el código.

In [4]:
nitrogen = {'name': 'nitrogen',
            'CAS': '7727-37-9',
            'atoms': ch.simple_formula_parser('N2'),
            'MW': ch.molecular_weight(ch.simple_formula_parser('N2')),
            'y_air': 0.79,
            'y_fuel': 0.0342,
            }

oxygen = {'name': 'oxygen',
          'CAS': '7782-44-7',
          'atoms': ch.simple_formula_parser('O2'),
          'MW': ch.molecular_weight(ch.simple_formula_parser('O2')),
          'y_air': 0.21,
          'y_fuel': 0,
          }

water = {'name': 'water',
         'CAS': '7732-18-5',
         'atoms': ch.simple_formula_parser('H2O'),
         'MW': ch.molecular_weight(ch.simple_formula_parser('H2O')),
         'y_air': 0.0,
         'y_fuel': 0,
         }

carbon_dioxide = {'name': 'carbon dioxide',
                  'CAS': '124-38-9',
                  'atoms': ch.simple_formula_parser('CO2'),
                  'MW': ch.molecular_weight(ch.simple_formula_parser('CO2')),
                  'y_air': 0.0,
                  'y_fuel': 0.0024,
                  }

#argon = {'name': 'argon',
#          'CAS': '7440-37-1',
#          'atoms': ch.simple_formula_parser('Ar'),
#          'MW': ch.molecular_weight(ch.simple_formula_parser('Ar')),
#          'y_air': 0,
#          'y_fuel': 0,
#          }

methane = {'name': 'methane',
           'CAS': '74-82-8',
           'atoms': ch.simple_formula_parser('CH4'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('CH4')),
           'y_air': 0,
           'y_fuel': 0.9008,
           }

ethane = {'name': 'ethane',
          'CAS': '74-84-0',
          'atoms': ch.simple_formula_parser('C2H6'),
          'MW': ch.molecular_weight(ch.simple_formula_parser('C2H6')),
          'y_air': 0,
          'y_fuel': 0.0473,
          }

propane = {'name': 'propane',
           'CAS' : '74-98-6',
           'atoms': ch.simple_formula_parser('C3H8'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C3H8')),
           'y_air': 0,
           'y_fuel': 0.0123,
           }

n_butane = {'name': 'n-butane',
           'CAS' : '106-97-8',
           'atoms': ch.simple_formula_parser('C4H10'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C4H10')),
           'y_air': 0,
           'y_fuel': 0.0024,
           }

n_pentane = {'name': 'n-pentane',
           'CAS' : '109-66-0',
           'atoms': ch.simple_formula_parser('C5H12'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C5H12')),
           'y_air': 0,
           'y_fuel': 0.0006,
           }

n_hexane = {'name': 'n-hexane',
           'CAS' : '110-54-3',
           'atoms': ch.simple_formula_parser('C6H14'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C6H14')),
           'y_air': 0,
           'y_fuel': 0.,
           }

n_heptane = {'name': 'n-heptane',
           'CAS' : '142-82-5',
           'atoms': ch.simple_formula_parser('C7H16'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C7H16')),
           'y_air': 0,
           'y_fuel': 0.,
           }

n_octane = {'name': 'n-octane',
           'CAS' : '111-65-9',
           'atoms': ch.simple_formula_parser('C8H18'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C8H18')),
           'y_air': 0,
           'y_fuel': 0.,
           }

n_nonane = {'name': 'n-nonane',
           'CAS' : '111-84-2',
           'atoms': ch.simple_formula_parser('C9H20'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C9H20')),
           'y_air': 0,
           'y_fuel': 0.,
           }

n_decane = {'name': 'n-decane',
           'CAS' : '124-18-5',
           'atoms': ch.simple_formula_parser('C10H22'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C10H22')),
           'y_air': 0,
           'y_fuel': 0.,
           }

i_butane = {'name': '2-methylpropane / i-butane',
           'CAS' : '75-28-5',
           'atoms': ch.simple_formula_parser('C4H10'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C4H10')),
           'y_air': 0,
           'y_fuel': 0.,
           }

i_pentane = {'name': '2-methylbutane / i-pentane',
           'CAS' : '78-78-4',
           'atoms': ch.simple_formula_parser('C5H12'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C5H12')),
           'y_air': 0,
           'y_fuel': 0.,
           }

ethylene = {'name': 'ethylene / ethene',
           'CAS' : '74-85-1',
           'atoms': ch.simple_formula_parser('C2H4'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C2H4')),
           'y_air': 0,
           'y_fuel': 0.,
           }

propylene = {'name': 'propylene / propene',
           'CAS' : '115-07-1',
           'atoms': ch.simple_formula_parser('C3H6'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C3H6')),
           'y_air': 0,
           'y_fuel': 0.,
           }

_1_butylene = {'name': '1-butylene / 1-butene',
           'CAS' : '106-98-9',
           'atoms': ch.simple_formula_parser('C4H8'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C4H8')),
           'y_air': 0,
           'y_fuel': 0.,
           }

cis_2_butylene = {'name': 'cis-2-butylene / cis-2-butene',
           'CAS' : '590-18-1',
           'atoms': ch.simple_formula_parser('C4H8'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C4H8')),
           'y_air': 0,
           'y_fuel': 0.,
           }

trans_2_butylene = {'name': 'trans-2-butylene / trans-2-butene',
           'CAS' : '624-64-6',
           'atoms': ch.simple_formula_parser('C4H8'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C4H8')),
           'y_air': 0,
           'y_fuel': 0.,
           }

i_butylene = {'name': 'i-butylene / i-butene',
           'CAS' : '115-11-7',
           'atoms': ch.simple_formula_parser('C4H8'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C4H8')),
           'y_air': 0,
           'y_fuel': 0.,
           }

_1_pentene = {'name': '1-pentene',
           'CAS' : '109-67-1',
           'atoms': ch.simple_formula_parser('C5H10'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('C5H10')),
           'y_air': 0,
           'y_fuel': 0.,
           }

carbon_monoxide = {'name': 'carbon monoxide',
           'CAS' : '630-08-0',
           'atoms': ch.simple_formula_parser('CO'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('CO')),
           'y_air': 0,
           'y_fuel': 0.,
           }

hydrogen = {'name': 'hydrogen',
           'CAS' : '1333-74-0',
           'atoms': ch.simple_formula_parser('H2'),
           'MW': ch.molecular_weight(ch.simple_formula_parser('H2')),
           'y_air': 0,
           'y_fuel': 0.,
           }

substances = [nitrogen,
              oxygen,
              water,
              carbon_dioxide,
              methane,
              ethane,
              propane,
              n_butane,
              n_pentane,
              n_hexane,
              n_heptane,
              n_octane,
              n_nonane,
              n_decane,
              i_butane,
              i_pentane,
              ethylene,
              propylene,
              _1_butylene,
              cis_2_butylene,
              trans_2_butylene,
              i_butylene,
              _1_pentene,
              carbon_monoxide,
              hydrogen]

names = [substances[i]['name'] for i in range(len(substances))]
MW = [substances[i]['MW'] for i in range(len(substances))]
CAS = [substances[i]['CAS'] for i in range(len(substances))]
atoms = [substances[i]['atoms'] for i in range(len(substances))]
zs_air = [substances[i]['y_air'] for i in range(len(substances))]
zs_fuel = [substances[i]['y_fuel'] for i in range(len(substances))]

Para concluir la carga de datos, debemos dar o bien variables intensivas, o bien variables extensivas. Elegimos el caudal de combustible y el exceso de aire utilizado.

In [5]:
n_fuel = 1.0     #Caudal molar en mol/s. Podemos cambiar la base para ponerlo en kg.
O2_excess = 4#0.18  #Exceso de aire.

# *Resolvedor*

Ejecutamos la resolución.

In [6]:
ans = ch.fuel_air_spec_solver(zs_air = zs_air,
                              zs_fuel = zs_fuel,
                              CASs = CAS,
                              atomss = atoms,
                              n_fuel = n_fuel,
                              O2_excess = O2_excess)

MW_air = np.dot(np.array(MW), np.array(zs_air))
MW_fuel = np.dot(np.array(MW), np.array(zs_fuel))
MW_gases = np.dot(np.array(MW), np.array(ans['zs_out']))

# *Resultados*

Las zs son las fracciones molares de los gases de combustión (en base húmeda). Usa el orden que establecimos más arriba.
```
zs_gases = [nitrogen,
            oxygen,
            water,
            carbon_dioxide,
            methane,
            ethane,
            propane,
            n_butane,
            n_pentane,
            n_hexane,
            n_heptane,
            n_octane,
            n_nonane,
            n_decane,
            i_butane,
            i_pentane,
            ethylene,
            propylene,
            _1_butylene,
            cis_2_butylene,
            trans_2_butylene,
            i_butylene,
            _1_pentene,
            carbon_monoxide,
            hydrogen]
```

In [7]:
[round(i, 5) for i in ans['zs_out']] #Redondeamos las fracciones molares de los gases de combustión.

[0.77419,
 0.16449,
 0.0403,
 0.02102,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

Obtenemos la fracción molar del oxígeno en los gases de combustión, tanto en base húmeda como en base seca.

In [8]:
round(ans['frac_out_O2'],5), round(ans['frac_out_O2_dry'],5)

(0.16449, 0.1714)

Obtenemos el caudal molar de aire necesario para realizar la combustión completa del caudal molar de combustible ingresado. Misma unidad que para este.

In [9]:
ans['n_air']

48.78690476190477

Obtenemos el caudal molar de los gases de combustión húmedos. Misma unidad que para el caudal molar de combustible ingresado.

In [10]:
[round(i, 5) for i in ans['ns_out']]

[38.57585,
 8.1962,
 2.0083,
 1.0473,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

Obtenemos las masas molares del aire, del combustible y de los gases húmedos. En [kg/kmol].

In [11]:
round(MW_air,2), round(MW_fuel,2), round(MW_gases,2)

(28.85, 17.66, 28.6)